In [1]:
from  bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time
import xlrd

url = 'https://www.domain.com.au/3-meldon-place-stanhope-gardens-nsw-2768-2014539320'

In [2]:
def get_property_sold_bysuburb(postcode = 2768):
    url = 'https://www.domain.com.au/sold-listings/?excludepricewithheld=1&ssubs=1&postcode={}'.format(postcode)
    page = 1
    master_list = []
    while page < 51:
        time.sleep(1)
        response = requests.get(url + '&page=' + str(page))
        soup = BeautifulSoup(response.text)

        for link in soup.find_all('li', 'css-1wd84zj'):
            try:
                product_url = link.find("a")["href"]
                #print(product_url)
                master_list.append(product_url)
            except:
                pass

        page = page + 1
    return master_list

In [3]:
def get_property_sold_bysuburb2(postcode = 2768):
    url = 'https://www.domain.com.au/sold-listings/?excludepricewithheld=1&ssubs=1&postcode={}'.format(postcode)
    page = 1
    master_list = []
    links_avail = True
    while links_avail:
        time.sleep(1)
        response = requests.get(url + '&page=' + str(page))
        soup = BeautifulSoup(response.text)
        if len(soup.find_all('li', 'css-1wd84zj'))==0:
                links_avail = False
        for link in soup.find_all('li', 'css-1wd84zj'):
            try:
                product_url = link.find("a")["href"]
                #print(product_url)
                master_list.append(product_url)
            except:
                pass

        page = page + 1
    return master_list

In [9]:
#get_property_sold_bysuburb(2768)
def download_house_details_by_url_list(list_of_houses_sold):
    house_data_list = []
    for each_house in list_of_houses_sold:
        time.sleep(1)
        house_data_list.append(get_property_details(each_house))
    return house_data_list


In [28]:
house_data_list = download_house_details_by_url_list(get_property_sold_bysuburb2(2170))


import pandas as pd
new_house_df = pd.DataFrame(house_data_list, columns=['property_id',
                                                   'Property_type',
                                                   'Price',
                                                   'Address',
                                                   'Description',
                                                   'date_sold',
                                                   'Beds',
                                                   'Bath',
                                                   'Parking',
                                                   'Property_Area',
                                                   'Agent_Names',
                                                   'Agency_Name',
                                                   'School_catchments',
                                                    'url'])
new_house_df.to_excel('C:\\Users\\sunda\OneDrive\\Documents\\Python\\RealEstate\\Downloaded Data\\House_price_data_2170.xlsx')

In [29]:
new_house_df.head()

,property_id,Property_type,Price,Address,Description,date_sold,Beds,Bath,Parking,Property_Area,Agent_Names,Agency_Name,School_catchments,url
0,2015122351,Apartment / Unit / Flat,"376,000","10/1 Riverpark Drive, Liverpool NSW 2170",Set in the perfect location within walking dis...,17 Jun 2019,2,1,2,0,[Michael Montes de Oca],Wiseberry Acclaim Prestons,"[Liverpool Public School\t0.7 km away\n, Liver...",https://www.domain.com.au/10-1-riverpark-drive...
1,2015289798,House,"717,000","218 Memorial Ave, Liverpool NSW 2170",A solid brick veneer home with plenty of stree...,15 Jun 2019,3,1,1,670,[Greg Andison],Century 21 Combined Liverpool,"[Liverpool West Public School\t0.5 km away\n, ...",https://www.domain.com.au/218-memorial-ave-liv...
2,2015292941,Vacant land,"520,000","8 Sheahan Street, Warwick Farm NSW 2170",Stake your claim in Liverpool's development bo...,15 Jun 2019,0,0,0,0,[Greg Andison],Century 21 Combined Liverpool,"[Warwick Farm Public School\t0.3 km away\n, Li...",https://www.domain.com.au/8-sheahan-street-war...
3,2015294942,Apartment / Unit / Flat,"480,000","Level 7, 39/65-69 Castlereagh Street, Liverpo...",Located in a PRIME position minutes away from ...,14 Jun 2019,2,2,2,0,"[Daniel Fletcher, Adrian Osorio]",Fletcher Property Group,"[Liverpool Public School\t0.6 km away\n, Liver...",https://www.domain.com.au/level-7-39-65-69-cas...
4,2015297451,House,"801,000","27 BARGO PLACE, Prestons NSW 2170",Approximately 9 years of age is this 28sq fami...,13 Jun 2019,4,3,2,0,[GAVIN BELCI],Starr Partners Liverpool,"[Dalmeny Public School\t0.8 km away\n, John Ed...",https://www.domain.com.au/27-bargo-place-prest...


In [ ]:
new_house_df

In [30]:
new_house_df.shape

(1000, 14)

In [11]:
def get_property_details(url):
    
    response_domain = requests.get(url)
    soup = BeautifulSoup(response_domain.text)
    property_details = [get_property_id(soup),
                        get_property_type(soup),
                        get_property_price(soup),
                        get_property_address(soup),
                        get_property_description(soup),
                        get_property_sale_date(soup),
                        get_property_bedroom_count(soup),
                        get_property_bathroom_count(soup),
                        get_property_parking_count(soup),
                        get_property_area(soup),
                        get_property_agent_names(soup),
                        get_property_agency(soup),
                        get_school_catchment(soup),
                        url
        
    ]
    
    return property_details

In [12]:
#get_property_details('https://www.domain.com.au/18-charlie-yankos-street-glenwood-nsw-2768-2015215396')

In [13]:
def get_property_id(soup):
    
    try:
        property_titles = soup.find('title')
        property_id = re.search(r'(\w{9,11})\s\|\sDomain', property_titles.text).group(1)
         
    except:
        property_id = 9999999999
    return property_id

In [14]:
def get_property_address(soup):
    try:
        address_container = soup.find('div',  attrs={'class': 'listing-details__summary-title-container'})
        address = address_container.find('h1').text
    except:
        address='NA'
    return address

In [15]:
def get_property_type(soup):
    try:
        property_type = soup.find('div', attrs={'class':'listing-details__key-features--value'}).text
    except:
        property_type='NA'
    return property_type

In [16]:
def get_property_price(soup):
    try:
        price_container = soup.find('div', attrs={'class':'listing-details__summary-title'}).text
        price = re.search(r'\$(.*)',price_container).group(1)
    except:
        price = 0
    
    return price

In [17]:
def get_property_description(soup):
    try:
        property_desc_container = soup.find_all('div',attrs={'class':'listing-details__description'})
        property_description = property_desc_container[0].find('p').text
    except:
        property_description = 'NA'
    return property_description

In [18]:
def get_property_sale_date(soup):
    try:
        property_titles = soup.find('title')
        sold_date = re.search(r'on\s(\d{2}\s\D{3}\s\d{4})', property_titles.text).group(1)
    except:
        sold_date = 'NA'
    return sold_date

In [19]:
def get_property_bedroom_count(soup):
    try:
        house_features = soup.find_all('span',attrs={'class':'property-feature__feature'})
        no_bedrooms = re.search(r'(\d)\s\w+',house_features[0].text).group(1)
    except:
        no_bedrooms = 0
    return no_bedrooms
    

In [20]:
def get_property_bathroom_count(soup):
    try:
        house_features = soup.find_all('span',attrs={'class':'property-feature__feature'})
        no_bathrooms = re.search(r'(\d)\s\w+',house_features[1].text).group(1)
    except:
        no_bathrooms = 0
    return no_bathrooms

In [21]:
def get_property_parking_count(soup):
    try:
        house_features = soup.find_all('span',attrs={'class':'property-feature__feature'})
        no_parking = re.search(r'(\d)\s\w+',house_features[2].text).group(1)
    except:
        no_parking = 0
    return no_parking

In [22]:
def get_property_area(soup):
    try:
        house_features = soup.find_all('span',attrs={'class':'property-feature__feature'})
        property_area = re.search(r'(\d+)',house_features[3].text).group(1)
    except:
        property_area = 0
    return property_area

In [23]:
def get_property_features(soup):
    try:
        property_features = []
        property_features_container = soup.find_all('li',attrs={'class':'listing-details__additional-features-listing'})
        for features in property_features_container:
            property_features.append(features.text)
    except:
         property_features = []
    return property_features

In [24]:
def get_get_property_type(soup):
    try:
        property_type = soup.find('div', attrs={'class':'listing-details__key-features--value'}).text
    except:
        property_type = 'NA'
    return property_type
        

In [25]:
def get_property_agent_names(soup):
    re_agent_names = []
    try:
        agents_details_container = soup.find_all('div', attrs={'class':'listing-details__agent-enquiry'})
        agent_names = agents_details_container[0].find_all('h3',attrs={'class':'listing-details__agent-enquiry-agent-name'})
        for agent_name in agent_names:
            re_agent_names.append(agent_name.text)
    
    except:
        agent_names = 'NA'
    return re_agent_names

In [26]:
def get_property_agency(soup):
    try:
        agents_details_container = soup.find_all('div', attrs={'class':'listing-details__agent-enquiry'})
        agency_name = agents_details_container[0].find('p', attrs={'class':'listing-details__agent-enquiry-agent-company-name'}).text
    except:
        agency_name = 'NA'
    return agency_name

In [27]:
def get_school_catchment(soup):
    schools_list = []
    try:
        school_container = soup.find_all('div',attrs={'class':re.compile('^school-catchment__school-summary-content')})
        for school in school_container:
            school_name = school.find('h5', attrs={'class':'school-catchment__school-title'})
            school_distance = school.find('div', attrs={'class':'school-catchment__school-distance'})
            schools_list.append(school_name.text + "\t" + school_distance.text + '\n')
    except:
        schools_list = []
    
    return schools_list